In [8]:
import pandas as pd
df = pd.read_csv("dataset_sintetico.csv")
print(df.columns)

# Transformação do DataFrame
# Certifique-se de que as colunas necessárias estão presentes
if 'Minuto_Exato' in df.columns:
    # Criar uma nova coluna para categorizar os minutos em faixas (se necessário)
    df['Minuto_Faixa'] = pd.cut(
        df['Minuto_Exato'],
        bins=[0, 30, 60, 90],
        labels=['0-30min', '31-60min', '61-90min'],
        right=False
    )

df.info()

Index(['Qtd_Cartoes', 'Minuto_Exato', 'Clube', 'Posicao', 'Taxa_Vitorias'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 939 entries, 0 to 938
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   Qtd_Cartoes    939 non-null    int64   
 1   Minuto_Exato   939 non-null    int64   
 2   Clube          939 non-null    object  
 3   Posicao        939 non-null    object  
 4   Taxa_Vitorias  939 non-null    float64 
 5   Minuto_Faixa   931 non-null    category
dtypes: category(1), float64(1), int64(2), object(2)
memory usage: 37.9+ KB


In [9]:
import plotly.express as px
import pandas as pd
import numpy as np

# Sunburst com cor representando o minuto da falta
fig = px.sunburst(df,
                  path=['Clube', 'Posicao', 'Minuto_Faixa'],
                  values='Qtd_Cartoes',
                  color='Minuto_Exato',  # 4ª VARIÁVEL COMO COR!
                  color_continuous_scale='Reds',
                  title='Sunburst: Clube → Posição → Período (Cor = Minuto da Falta)',
                  hover_data=['Minuto_Exato', 'Qtd_Cartoes'])

fig.update_traces(textinfo='label+percent parent')
fig.update_layout(height=700)
fig.show()


ValueError: ('Non-leaves rows are not permitted in the dataframe \n', ('Clube 11', 'Atacante', ''), 'is not a leaf.')

In [12]:
import plotly.express as px
import pandas as pd
import numpy as np

# Dados agregados por todas as 4 variáveis
np.random.seed(42)
data = []

clubes = [f'Clube_{i}' for i in range(1, 8)]
posicoes = ['Goleiro', 'Zagueiro', 'Meio-campo', 'Atacante', 'Lateral']
periodos = ['0-30min', '31-60min', '61-90min']

for clube in clubes:
    for posicao in posicoes:
        for periodo in periodos:
            data.append({
                'Clube': clube,
                'Posicao': posicao,
                'Periodo': periodo,
                'Qtd_Cartoes': np.random.randint(1, 12)
            })

df = pd.DataFrame(data)

# Sunburst com 4 níveis: Centro → Clube → Posição → Minuto
fig = px.sunburst(df,
                  path=['Clube', 'Posicao', 'Periodo'],  # 3 níveis hierárquicos
                  values='Qtd_Cartoes',  # Tamanho dos setores
                  color='Qtd_Cartoes',  # Cor = quantidade de cartões
                  color_continuous_scale='YlOrRd',
                  title='4 Variáveis: Clube → Posição → Período do Minuto (tamanho e cor = cartões)',
                  hover_data=['Qtd_Cartoes'])

fig.update_traces(textinfo='label+value')
fig.update_layout(height=700, width=900)
fig.show()


In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

fig = go.Figure(go.Sunburst(
    ids=df['ids'],
    labels=df['labels'],
    parents=df['parents'],
    values=df['values'],
    marker=dict(
        colors=df['minuto_medio'],
        colorscale='RdYlGn_r',
        colorbar=dict(
            title='Minuto<br>da Falta',
            thickness=20,
            len=0.7
        )
    ),
    text=df['minuto_medio'].round(0).astype(str) + 'min',
    hovertemplate='<b>%{label}</b><br>Cartões: %{value}<br>Minuto: %{color:.0f}<extra></extra>'
))

fig.update_layout(
    title='Clube → Período do Jogo (cor e texto = minuto médio das faltas)',
    height=700,
    width=900
)

fig.show()


In [11]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd

# Preparar dados
np.random.seed(42)
n_points = 60

# Calcular ângulo baseado no minuto (0-90 min = 0-360 graus)
df['angulo'] = (df['minuto'] / 90) * 360

# Calcular raio baseado em clube e posição
df['raio_base'] = df['clube_id'] * 0.8  # Cada clube em um anel diferente
df['raio'] = df['raio_base'] + (df['posicao_id'] * 0.1)  # Posição dentro do anel

# Converter para coordenadas cartesianas
df['x'] = df['raio'] * np.cos(np.radians(df['angulo']))
df['y'] = df['raio'] * np.sin(np.radians(df['angulo']))

# Criar gráfico
fig = go.Figure()

# Plotar pontos por posição
for posicao in df['posicao'].unique():
    df_pos = df[df['posicao'] == posicao]
    
    fig.add_trace(go.Scatter(
        x=df_pos['x'],
        y=df_pos['y'],
        mode='markers',
        name=posicao,
        marker=dict(
            size=df_pos['cartoes'] * 3,  # Tamanho = quantidade de cartões
            color=df_pos['minuto'],  # Cor = minuto
            colorscale='Reds',
            showscale=True,
            colorbar=dict(title='Minuto'),
            line=dict(width=1, color='DarkSlateGray')
        ),
        text=df_pos['clube'] + '<br>' + df_pos['posicao'] + '<br>Min: ' + 
             df_pos['minuto'].astype(str) + '<br>Cartões: ' + df_pos['cartoes'].astype(str),
        hoverinfo='text'
    ))

# Adicionar círculos de referência para clubes
for i in range(1, 11):
    theta = np.linspace(0, 2*np.pi, 100)
    r = i * 0.8
    x_circle = r * np.cos(theta)
    y_circle = r * np.sin(theta)
    
    fig.add_trace(go.Scatter(
        x=x_circle,
        y=y_circle,
        mode='lines',
        line=dict(color='lightgray', width=1, dash='dash'),
        showlegend=False,
        hoverinfo='skip'
    ))

# Adicionar linhas radiais para minutos
for minuto in [0, 15, 30, 45, 60, 75, 90]:
    angulo = (minuto / 90) * 360
    x_line = [0, 10 * np.cos(np.radians(angulo))]
    y_line = [0, 10 * np.sin(np.radians(angulo))]
    
    fig.add_trace(go.Scatter(
        x=x_line,
        y=y_line,
        mode='lines+text',
        line=dict(color='lightgray', width=1),
        text=['', f'{minuto}min'],
        textposition='top center',
        showlegend=False,
        hoverinfo='skip'
    ))

fig.update_layout(
    title='Gráfico Radial: 4 Variáveis<br>(Ângulo=Minuto, Raio=Clube, Cor=Minuto, Tamanho=Cartões, Símbolo=Posição)',
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    height=800,
    width=900,
    plot_bgcolor='white'
)

fig.show()


KeyError: 'minuto'

In [6]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Dados de exemplo
np.random.seed(42)
clubes = [f'Clube_{i}' for i in range(1, 9)]
posicoes = ['Goleiro', 'Zagueiro', 'Meio-campo', 'Atacante', 'Lateral']

ids = []
labels = []
parents = []
values = []
minutos = []  # 4ª variável!

# Raiz
ids.append('Total')
labels.append('Total')
parents.append('')
values.append(0)
minutos.append(45)

# Nível 1: Clubes
for clube in clubes:
    ids.append(clube)
    labels.append(clube)
    parents.append('Total')
    values.append(np.random.randint(10, 30))
    minutos.append(np.random.randint(30, 60))

# Nível 2: Posições
for clube in clubes:
    for posicao in posicoes:
        id_pos = f'{clube}-{posicao}'
        ids.append(id_pos)
        labels.append(posicao)
        parents.append(clube)
        values.append(np.random.randint(2, 8))
        minutos.append(np.random.randint(0, 90))  # Minuto médio da falta

df = pd.DataFrame({
    'ids': ids,
    'labels': labels,
    'parents': parents,
    'values': values,
    'minuto_medio': minutos
})

# Criar escala de cores baseada no minuto
df['color_value'] = df['minuto_medio']

fig = go.Figure(go.Sunburst(
    ids=df['ids'],
    labels=df['labels'],
    parents=df['parents'],
    values=df['values'],
    marker=dict(
        colors=df['color_value'],
        colorscale='Reds',
        cmid=45,
        colorbar=dict(title='Minuto<br>Médio')
    ),
    branchvalues='total',
    hovertemplate='<b>%{label}</b><br>Cartões: %{value}<br>Minuto médio: %{color}<extra></extra>'
))

fig.update_layout(
    title='4 Variáveis: Clube → Posição (cor = minuto médio da falta)',
    height=700,
    width=900
)

fig.show()
